In [85]:
import pip
import numpy as np
import pandas as pd
import sklearn
import tabulate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from sklearn.metrics import classification_report,confusion_matrix



In [3]:
song_app = pd.read_csv("../../SongApp.csv",header=-1,)

In [4]:
song_app.head()

,0,1,2,3,4,5,6,7,8,9,...,82,83,84,85,86,87,88,89,90,91
0,NaN,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,0.0,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,1.0,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2.0,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,3.0,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [5]:
song_app.size

42661872

In [6]:
song_app.shape

(463716, 92)

In [7]:
# clean data
df_song = song_app.dropna()
df_song.__delitem__(0)

In [8]:
df_song.head()

,1,2,3,4,5,6,7,8,9,10,...,82,83,84,85,86,87,88,89,90,91
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
5,2001,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [9]:
labels_column = df_song.iloc[0:,0]
labels = df_song.values[0:,0]
labels

array([2001., 2001., 2001., ..., 2006., 2007., 2007.])

In [10]:
df_song.__delitem__(1)

In [11]:
df_song['labels'] = labels_column

/Users/neil/Code/song_classifier/venv/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
num_columns = df_song.columns.shape[0]
columns = list(range(num_columns - 1))
columns_name = list(map(str,columns)) + ['labels']
df_song.columns = columns_name
df_song.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,labels
1,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,...,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061,2001
2,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,...,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345,2001
3,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,...,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382,2001
4,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,...,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903,2001
5,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,-4.69088,...,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263,2001


# LET'S BALANCE THE DATASET

In [151]:
# value_count = song_app['labels'].value_counts()
df = df_song
# sort the dataframe
df.sort_values('labels', inplace=True)
# set the index to be this and don't drop
df.set_index(keys=['labels'], drop=False,inplace=True)
# get a list of names
labels=df['labels'].unique().tolist()
# now we can perform a lookup on a 'view' of the dataframe

df_list = []
for label in labels:
    label_df = df.loc[df['labels'] == label]
    df_list.append(label_df)

# joe = df.loc[df['labels'] == 2008]


# nb_instance_target = 5210
nb_instance_target = 15210

# print('before reseampling:')
# print df_list[88]
# print(len(df_list[88]))

new_df_list = []
for df in df_list:
    if len(df) > nb_instance_target:
        df_majority_downsampled = resample(df, 
                                 replace=False,    # sample without replacement
                                 n_samples=nb_instance_target,     # to match minority class
                                 random_state=123)
        new_df_list.append(df_majority_downsampled)
    elif len(df) < nb_instance_target:
        df_minority_upsampled = resample(df, 
                                 replace=True,     # sample with replacement
                                 n_samples= nb_instance_target,    # to match majority class
                                 random_state=123)
        new_df_list.append(df_minority_upsampled)

# print('After reseampling:')
# print(new_df_list[88])
# print(len(new_df_list[88]))   

df_resampled = pd.concat(new_df_list)
df_resampled = df_resampled.reset_index(drop=True)
print (song_app.shape)

print (df_resampled.shape)


/Users/neil/Code/song_classifier/venv/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


(463716, 92)
(1353690, 91)


# TEST WITH BALANCED

In [156]:
x_data = df_resampled.sample(100000)
# x_data = df_resampled

labels = x_data['labels']
x_data = x_data.iloc[:, 0:89]
# x_data.head()
labels

501877     1955
788813     1974
43537      1925
747581     1972
1133550    1997
530246     1957
230316     1938
782694     1974
1108411    1995
823731     1977
767327     1973
175772     1934
466276     1953
1278054    2007
115944     1930
931311     1984
225379     1937
356624     1946
955986     1985
393647     1948
1341161    2011
1338176    2010
1003240    1988
373219     1947
572183     1960
710986     1969
1256310    2005
706948     1969
804436     1975
283932     1941
           ... 
353563     1946
854937     1979
182        1922
470773     1953
1267399    2006
242007     1938
146957     1932
788742     1974
736162     1971
747272     1972
814716     1976
935176     1984
43259      1925
37178      1925
968002     1986
499140     1955
43727      1925
975877     1987
237300     1938
1281581    2007
105996     1929
318499     1943
420136     1950
375273     1947
725301     1970
646253     1965
1093308    1994
259313     1940
390244     1948
706385     1969
Name: labels, Length: 10

In [157]:
# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(90000, 89)
(10000, 89)
(90000,)
(10000,)


In [130]:
from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train)
# y_train_scaled = preprocessing.scale(y_train)

In [158]:
clf = RandomForestRegressor(n_jobs=5, n_estimators=50)
model = clf.fit(X_train, y_train)

In [159]:
y_predict_test = model.predict(X_test)

In [160]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error

In [161]:
def log_errors(y_test, predicted):
    mae = mean_absolute_error(y_test, predicted)
    mse = mean_squared_error(y_test, predicted, multioutput='raw_values')
    r2 = r2_score(y_test, predicted)
    medae = median_absolute_error(y_test, predicted)
    print('mae ',mae)
    print('mse ',mse)
    print('r2 score ', r2)
    print('median_ae ',medae)

In [162]:
log_errors(y_test=y_test,predicted=y_predict_test)

('mae ', 4.78757)
('mse ', array([69.38571708]))
('r2 score ', 0.8948279279223452)
('median_ae ', 1.5199999999999818)


In [163]:
list_test = np.around(y_test)

# list_predict = predict_y.astype('int')
list_predict = np.around(y_predict_test)

# print list_test[0]
# print list_predict[0]
print(classification_report(list_predict, list_test ))

             precision    recall  f1-score   support

     1922.0       1.00      1.00      1.00       107
     1924.0       1.00      1.00      1.00       127
     1925.0       1.00      1.00      1.00       122
     1926.0       1.00      1.00      1.00       108
     1927.0       1.00      1.00      1.00        99
     1928.0       1.00      1.00      1.00       123
     1929.0       1.00      1.00      1.00       123
     1930.0       1.00      1.00      1.00       108
     1931.0       1.00      1.00      1.00       107
     1932.0       1.00      1.00      1.00       105
     1933.0       1.00      1.00      1.00        97
     1934.0       1.00      1.00      1.00       119
     1935.0       1.00      1.00      1.00       104
     1936.0       1.00      1.00      1.00       120
     1937.0       1.00      1.00      1.00       125
     1938.0       1.00      1.00      1.00        89
     1939.0       1.00      1.00      1.00       101
     1940.0       1.00      1.00      1.00   

In [149]:
# x_data = df_resampled.reset_index(drop=True)
x_data = df_song

# x_pif = df_resampled.sample(500)
x_pif = x_data

y_pif = x_pif['labels']

x_pif = x_pif.iloc[:, 0:89]



y_predict_pif = model.predict(x_pif)

log_errors(y_test=y_pif,predicted=y_predict_pif)

('mae ', 7.17625131815878)
('mse ', array([94.27049857]))
('r2 score ', 0.21230140108970763)
('median_ae ', 5.2999999999999545)


In [150]:
list_test = np.around(y_pif)

# list_predict = predict_y.astype('int')
list_predict = np.around(y_predict_pif)

# print list_test[0]
# print list_predict[0]
print(classification_report(list_predict, list_test ))

             precision    recall  f1-score   support

     1922.0       1.00      1.00      1.00         6
     1924.0       1.00      1.00      1.00         5
     1925.0       1.00      1.00      1.00         7
     1926.0       1.00      1.00      1.00        19
     1927.0       1.00      1.00      1.00        40
     1928.0       1.00      1.00      1.00        48
     1929.0       1.00      0.99      0.99        80
     1930.0       1.00      1.00      1.00        38
     1931.0       1.00      0.97      0.98        32
     1932.0       1.00      0.92      0.96        12
     1933.0       1.00      0.86      0.92         7
     1934.0       1.00      1.00      1.00        28
     1935.0       1.00      1.00      1.00        24
     1936.0       1.00      0.92      0.96        24
     1937.0       1.00      0.93      0.96        27
     1938.0       1.00      0.95      0.97        20
     1939.0       1.00      1.00      1.00        35
     1940.0       1.00      0.88      0.93   

# TEST WITH UNBALANCED

In [9]:
x_data = df_song
x_data.__delitem__('labels')

In [62]:
x_data.head()

,0,1,2,3,4,5,6,7,8,9,...,80,81,82,83,84,85,86,87,88,89
1,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,4.59210,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,1.39518,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,-6.36304,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,0.93609,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903
5,50.54767,0.31568,92.35066,22.38696,-25.51870,-19.04928,20.67345,-5.19943,3.63566,-4.69088,...,6.59753,-50.69577,26.02574,18.94430,-0.33730,6.09352,35.18381,5.00283,-11.02257,0.02263


In [10]:
# split into training and test data
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size=0.1, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(417343, 90)
(46372, 90)
(417343,)
(46372,)


In [75]:
from sklearn import preprocessing
X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = preprocessing.scale(y_train)

In [12]:
clf = RandomForestRegressor(n_jobs=5, n_estimators=50)
model = clf.fit(X_train_scaled, y_train_scaled)

In [17]:
X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = preprocessing.scale(y_test)
predicted = model.predict(X_test_scaled)

In [14]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error

In [15]:
def log_errors(y_test, predicted):
    mae = mean_absolute_error(y_test, predicted)
    mse = mean_squared_error(y_test, predicted, multioutput='raw_values')
    r2 = r2_score(y_test, predicted)
    medae = median_absolute_error(y_test, predicted)
    print('mae ',mae)
    print('mse ',mse)
    print('r2 score ', r2)
    print('median_ae ',medae)

In [19]:
log_errors(y_test=y_test_scaled,predicted=predicted)

mae  0.593008904188
mse  [ 0.69729548]
r2 score  0.30270452098
median_ae  0.419795584823
